Sentiment analysis

In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

# Step 3: Load IMDB dataset for sentiment analysis (pre-built dataset in Keras)
from tensorflow.keras.datasets import imdb

# Load dataset and limit the vocabulary to the top 10000 most frequent words
vocab_size = 10000
maxlen = 100  # Maximum length of reviews

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Step 4: Pad sequences to ensure consistent input size for the model
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Step 5: Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Step 6: Build the sentiment analysis model
model = models.Sequential()
model.add(layers.Embedding(vocab_size, 128, input_length=maxlen))
model.add(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

# Step 7: Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 8: Train the model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_val, y_val))

# Step 9: Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Step 10: Predict sentiment on new data
sample_text = "This movie was amazing! I loved the characters and the plot was great."
# Tokenize the input text (the tokenizer for IMDB is the same as the one used for training data)
word_index = imdb.get_word_index()
tokens = sample_text.lower().split()
encoded = [word_index.get(word, 0) for word in tokens]  # Convert words to their index values
encoded = pad_sequences([encoded], maxlen=maxlen)  # Pad the sequence

# Predict sentiment
prediction = model.predict(encoded)
print(f"Sentiment: {'Positive' if prediction >= 0.5 else 'Negative'}")


Spelling correction

In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow
!pip install nltk

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import nltk
from nltk.corpus import words
from sklearn.model_selection import train_test_split
import numpy as np
import random
import re

# Download NLTK data
nltk.download('words')

# Step 3: Load and preprocess the dataset
# We'll use a simple dataset of common misspellings and their correct words

# Sample data (misspelled words -> correct words)
corrections = [
    ("recieve", "receive"),
    ("adn", "and"),
    ("teh", "the"),
    ("definately", "definitely"),
    ("occured", "occurred"),
    ("untill", "until"),
    ("seperated", "separated"),
    ("alot", "a lot"),
    ("arguement", "argument"),
    ("beleive", "believe")
]

# Split the data into training and testing sets
random.shuffle(corrections)
train_data = corrections[:int(0.8*len(corrections))]
test_data = corrections[int(0.8*len(corrections)):]

# Step 4: Tokenize the words (characters) and prepare input-output sequences
input_texts = [pair[0] for pair in train_data]
target_texts = ['\t' + pair[1] + '\n' for pair in train_data]  # Add start and end token

# Tokenization
input_chars = sorted(set(''.join(input_texts)))
target_chars = sorted(set(''.join(target_texts)))
input_char_index = {char: i for i, char in enumerate(input_chars)}
target_char_index = {char: i for i, char in enumerate(target_chars)}
index_input_char = {i: char for char, i in input_char_index.items()}
index_target_char = {i: char for char, i in target_char_index.items()}

# Step 5: Vectorize the data (convert to integer sequences)
max_input_length = max([len(txt) for txt in input_texts])
max_target_length = max([len(txt) for txt in target_texts])

encoder_input_data = np.zeros((len(input_texts), max_input_length), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_target_length), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_target_length, len(target_chars)), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_char_index[char]
    for t, char in enumerate(target_text):
        decoder_input_data[i, t] = target_char_index[char]
        if t > 0:
            decoder_target_data[i, t - 1, target_char_index[char]] = 1.

# Step 6: Build the seq2seq model
latent_dim = 256

# Encoder
encoder_inputs = layers.Input(shape=(None,))
encoder_embedding = layers.Embedding(len(input_chars), latent_dim)(encoder_inputs)
encoder_lstm = layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = layers.Input(shape=(None,))
decoder_embedding = layers.Embedding(len(target_chars), latent_dim)(decoder_inputs)
decoder_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = layers.Dense(len(target_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Step 7: Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100, validation_split=0.2)

# Step 9: Save the model
model.save("spelling_correction_model.h5")

# Step 10: Evaluate the model (on test set)
input_texts_test = [pair[0] for pair in test_data]
target_texts_test = ['\t' + pair[1] + '\n' for pair in test_data]
encoder_input_data_test = np.zeros((len(input_texts_test), max_input_length), dtype='float32')
decoder_input_data_test = np.zeros((len(input_texts_test), max_target_length), dtype='float32')
decoder_target_data_test = np.zeros((len(input_texts_test), max_target_length, len(target_chars)), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts_test, target_texts_test)):
    for t, char in enumerate(input_text):
        encoder_input_data_test[i, t] = input_char_index[char]
    for t, char in enumerate(target_text):
        decoder_input_data_test[i, t] = target_char_index[char]
        if t > 0:
            decoder_target_data_test[i, t - 1, target_char_index[char]] = 1.

test_loss, test_acc = model.evaluate([encoder_input_data_test, decoder_input_data_test], decoder_target_data_test)
print(f"Test accuracy: {test_acc:.4f}")

# Step 11: Predict spelling correction on new data
def predict_spelling_correction(input_text):
    encoder_input_seq = np.zeros((1, max_input_length), dtype='float32')
    for t, char in enumerate(input_text):
        encoder_input_seq[0, t] = input_char_index.get(char, 0)

    states_value = model.predict(encoder_input_seq)

    # Prepare the decoder input
    target_seq = np.zeros((1, 1), dtype='float32')
    target_seq[0, 0] = target_char_index['\t']

    # Generate the output sequence
    decoded_sentence = ''
    for _ in range(max_target_length):
        output_tokens = model.predict([encoder_input_seq, target_seq])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_target_char[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        # Update the target sequence
        target_seq[0, 0] = sampled_token_index

    return decoded_sentence.strip()

# Example usage
input_word = "recieve"
corrected_word = predict_spelling_correction(input_word)
print(f"Corrected Word: {corrected_word}")


Word sense disambiguation

In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow
!pip install nltk
!pip install gensim

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
import nltk
from nltk.corpus import wordnet as wn
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import Word2Vec

# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('semcor')
nltk.download('punkt')

# Step 3: Load SemCor dataset for word sense disambiguation (WSD)
from nltk.corpus import semcor

# Sample sentence from SemCor (context with sense-tagged word)
context = semcor.sents()[:1000]  # Use first 1000 sentences for training

# Step 4: Preprocessing and extracting word-context pairs
word_contexts = []
sense_labels = []

for sentence in context:
    for word in sentence:
        if isinstance(word, tuple):  # Word with sense tag (e.g., ('dog', 'n'))
            word_token, sense_tag = word
            word_contexts.append(sentence)
            sense_labels.append(sense_tag)

# Step 5: Prepare word embeddings using Word2Vec for context representation
# Using Word2Vec to create word embeddings for each word in the context
model_w2v = Word2Vec(sentences=[nltk.word_tokenize(' '.join(sent)) for sent in context], vector_size=100, window=5, min_count=1, sg=0)

# Step 6: Convert word-context pairs into feature vectors
def get_word_embedding(word):
    try:
        return model_w2v.wv[word]
    except KeyError:
        return np.zeros(100)

X = np.array([np.mean([get_word_embedding(word) for word in sent if isinstance(word, str)], axis=0) for sent in word_contexts])
y = np.array([wn.synset(sense).lemmas()[0].key() for sense in sense_labels])

# Step 7: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Build a simple Neural Network model for WSD
model = models.Sequential()
model.add(layers.Input(shape=(100,)))  # Input layer (word embeddings)
model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dense(len(set(y)), activation='softmax'))  # Output layer (classification)

# Step 9: Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 10: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Step 11: Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Step 12: Predict the word sense for new context
def predict_word_sense(context_sentence):
    sentence_embedding = np.mean([get_word_embedding(word) for word in nltk.word_tokenize(context_sentence)], axis=0).reshape(1, -1)
    prediction = model.predict(sentence_embedding)
    sense_index = np.argmax(prediction)
    return wn.lemmas()[sense_index].name()  # Get the sense name from WordNet

# Example usage: Predict the word sense for a new sentence
context_sentence = "The bank was full of fish."
predicted_sense = predict_word_sense(context_sentence)
print(f"Predicted Sense: {predicted_sense}")


Machine translation

In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow
!pip install nltk

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
from nltk.translate import bleu_score

# Step 3: Download and preprocess data (English-French translation pairs)
nltk.download('translate')

# Example parallel corpus (English-French translation)
english_sentences = [
    'hello', 'how are you', 'good morning', 'good evening', 'good night',
    'thank you', 'goodbye', 'please', 'sorry', 'excuse me'
]

french_sentences = [
    'bonjour', 'comment ça va', 'bon matin', 'bonsoir', 'bonne nuit',
    'merci', 'au revoir', 's’il vous plaît', 'désolé', 'excusez-moi'
]

# Step 4: Tokenization and creating word indices
input_texts = english_sentences
output_texts = ['\t' + sentence + '\n' for sentence in french_sentences]  # Adding start and end token

input_chars = sorted(set(''.join(input_texts)))
output_chars = sorted(set(''.join(output_texts)))
input_char_index = {char: i for i, char in enumerate(input_chars)}
output_char_index = {char: i for i, char in enumerate(output_chars)}
index_input_char = {i: char for char, i in input_char_index.items()}
index_output_char = {i: char for char, i in output_char_index.items()}

# Step 5: Vectorizing the data (convert to integer sequences)
max_input_length = max([len(txt) for txt in input_texts])
max_output_length = max([len(txt) for txt in output_texts])

encoder_input_data = np.zeros((len(input_texts), max_input_length), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_output_length), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_output_length, len(output_chars)), dtype='float32')

for i, (input_text, output_text) in enumerate(zip(input_texts, output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t] = input_char_index[char]
    for t, char in enumerate(output_text):
        decoder_input_data[i, t] = output_char_index[char]
        if t > 0:
            decoder_target_data[i, t - 1, output_char_index[char]] = 1.

# Step 6: Build Seq2Seq model with attention mechanism
latent_dim = 256

# Encoder
encoder_inputs = layers.Input(shape=(None,))
encoder_embedding = layers.Embedding(len(input_chars), latent_dim)(encoder_inputs)
encoder_lstm = layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = layers.Input(shape=(None,))
decoder_embedding = layers.Embedding(len(output_chars), latent_dim)(decoder_inputs)
decoder_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = layers.Dense(len(output_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Step 7: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100, validation_split=0.2)

# Step 9: Evaluate the model (on test set)
test_loss, test_acc = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data)
print(f"Test accuracy: {test_acc:.4f}")

# Step 10: Predict translation on new data
def predict_translation(input_text):
    encoder_input_seq = np.zeros((1, max_input_length), dtype='float32')
    for t, char in enumerate(input_text):
        encoder_input_seq[0, t] = input_char_index.get(char, 0)

    states_value = model.predict(encoder_input_seq)

    # Prepare the decoder input
    target_seq = np.zeros((1, 1), dtype='float32')
    target_seq[0, 0] = output_char_index['\t']

    # Generate the output sequence
    decoded_sentence = ''
    for _ in range(max_output_length):
        output_tokens = model.predict([encoder_input_seq, target_seq])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_output_char[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n':
            break

        # Update the target sequence
        target_seq[0, 0] = sampled_token_index

    return decoded_sentence.strip()

# Example usage: Translate a new sentence from English to French
input_sentence = "hello"
predicted_translation = predict_translation(input_sentence)
print(f"Predicted Translation: {predicted_translation}")


Text Classification

In [ ]:
# Step 1: Install necessary libraries
!pip install tensorflow
!pip install nltk

# Step 2: Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Step 3: Load the IMDB dataset
# IMDB dataset is a built-in dataset in TensorFlow/Keras (reviews and sentiment labels)
num_words = 10000  # Limit to top 10,000 most frequent words

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Step 4: Pad sequences to ensure uniform input length
max_len = 500  # Maximum sequence length (we'll truncate/pad sequences to this length)
x_train_pad = pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
x_test_pad = pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')

# Step 5: Build the Text Classification Model (LSTM-based)
model = models.Sequential()

# Embedding layer to learn word representations
model.add(layers.Embedding(input_dim=num_words, output_dim=128, input_length=max_len))

# LSTM layer for sequence processing
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# Dense layer for classification (output is binary: positive/negative)
model.add(layers.Dense(1, activation='sigmoid'))

# Step 6: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 7: Train the model
model.fit(x_train_pad, y_train, epochs=5, batch_size=64, validation_data=(x_test_pad, y_test))

# Step 8: Evaluate the model
test_loss, test_acc = model.evaluate(x_test_pad, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# Step 9: Predict sentiment of a new review
def predict_sentiment(review):
    # Tokenize and pad the review text
    review_seq = imdb.get_word_index()
    review_tokenized = [review_seq.get(word, 0) for word in review.split()]
    review_padded = pad_sequences([review_tokenized], maxlen=max_len, padding='post', truncating='post')

    # Predict sentiment (0 for negative, 1 for positive)
    sentiment = model.predict(review_padded)
    return "Positive" if sentiment >= 0.5 else "Negative"

# Example usage: Predict sentiment for a new review
review_text = "This movie was fantastic! I really enjoyed it."
predicted_sentiment = predict_sentiment(review_text)
print(f"Predicted Sentiment: {predicted_sentiment}")


Question answering system.

In [ ]:
# Step 1: Install necessary libraries
!pip install transformers
!pip install torch
!pip install datasets

# Step 2: Import necessary libraries
from transformers import pipeline, BertTokenizer, BertForQuestionAnswering
import torch

# Step 3: Load the pre-trained BERT model and tokenizer for Question Answering
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Step 4: Create the Question Answering pipeline
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)

# Step 5: Define context (passage) and a sample question
context = """
Bert (Bidirectional Encoder Representations from Transformers) is a transformer-based model designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers. The model has achieved state-of-the-art results on a wide array of NLP tasks, including Question Answering (QA), Language Inference (LI), and Named Entity Recognition (NER). The BERT model is trained using large corpora and fine-tuned on smaller datasets to adapt to specific tasks.
"""

question = "What tasks has BERT achieved state-of-the-art results on?"

# Step 6: Use the model to answer the question
result = qa_pipeline({'context': context, 'question': question})

# Step 7: Output the result (answer)
print(f"Answer: {result['answer']}")
